# About

Within the context of the CERISE project we have created several sreadsheets that list different entities and their properties. 
This code aims to automate the extraction of data from those spreadsheets to render them as pages in a quarto website. 

NOTE: 
I wasn't able to download the relevant online data with code. Instead, I download the spreadsheets manually as csv files (stored in the data folder).
In a future iteration the website generation could be fully automated using CI.



In [1]:
import shutil
import os
import pandas as pd


# Custom functions

In [2]:
def tidy_filename(filename):
    return filename.strip().replace("'", '_')

In [3]:

def save_qmd(content, filename):
    
    # tidy filename
    filename = tidy_filename(filename)
    # write documentation into qmd file
    md_file = open(filename + '.qmd', 'w')
    md_file.write(content.encode('utf8'))
    md_file.close()



In [4]:
# loading of the data from spreadsheet does not work because of csv formatting issue.


## Instructions google sheet:
# document_id = '1FE-NipeF_ns5yoiXXi0mHm0FF21VYoJfC5recSE441s'
# sheet_name = 'initiatives'
# sheet_url = f"https://docs.google.com/spreadsheets/d/{document_id}/gviz/tq?tqx=out:json&sheet={sheet_name}"

# initiatives = pd.read_csv(sheet_url)


In [9]:
# describes how data from the spreadsheet is converted to a page:

def convert_row_to_qmd(row, output_dir, usecase):

    if usecase != 'initiative':
            
        text = f'## {row["Name"]}\n'
        text += f'### Description\n\n{row["Description"]}\n\n'
        
        filename = row["Name"].lower().replace(" ", "-")
        
    else:
        
        text = f'## {row["name"]}\n'
        text += f'### Description\n\n{row["description"]}\n\n'
        
        filename = row["name"].lower().replace(" ", "-")

        

        text += f'### Type of Initiative\n\n{row["type"]}\n\n'
        text += f'### Provider\n\n{row["provider_name"]}\n\n'
        text += f'### Creator\n\n{row["creator_name"]}\n\n'
        

    # loop over variables to generate output if it variables have been entered.
    vars = row.index[6:]
    for v in vars:

        if (not pd.isna(row[v])):

            text += f'### {v}\n\n{row[v]}\n\n' # replace v by v_title to get from a separate table



    

    filename =  output_dir + filename

    save_qmd(text, filename)

    return text






In [10]:
def extract_data_for_website(data_filename, output_dir, data_page, usecase):
    
    
    
    # importing data 
    df = pd.read_csv(data_filename, encoding = "ISO-8859-1")
        
    # loop over rows and save each row as qmd file
    for i in range(len(df)):
        row = df.loc[i,]
        convert_row_to_qmd(row, output_dir, usecase)
    
    # list files in the output folder (should yeield all the auto-generated qmd files):
    auto_files = os.listdir(output_dir)
    
    # add included statements in the initiative file
    text = '' 
    text += '\n\n'

    for i in auto_files:
        text += '{{'
        text += f'< include {output_dir}{i} >'
        text += '}}\n\n'

    save_qmd(text, data_page)


# Data extraction and conversion


## 1. List of green skills initiatives


In [11]:
usecases = [
                {
                    'input_file': 'data/WP2A1-Greenskills initiatives - initiatives.csv',
                    'output_dir': 'green-skills-initiatives/auto-generated/',
                    'data_page':  'green-skills-initiatives-list',
                    'usecase': 'initiative'
                },
                {
                    'input_file': 'data/WP2A2 - Green Skills Resources(Green Skills Resources).csv',
                    'output_dir': 'green-skills-resources/auto-generated/',
                    'data_page': 'green-skills-resources-list',
                    'usecase': 'resource'
                }
            ]


In [12]:
for u in usecases:
    # process the input data
    extract_data_for_website(u['input_file'], 
                             u['output_dir'],
                             u['data_page'],
                             u['usecase'])
    
    

In [ ]:
usecases[0]['a']

In [8]:


# importing data from manually downloaded spreadsheet
input_data = pd.read_csv('data/WP2A1-Greenskills initiatives - initiatives.csv')

# location to store the auto-generated qmd files 
output_dir = 'green-skills-initiatives/auto-generated/' 

# there is an intro page for that spreadsheet
data_page = 'green-skills-initiatives-list'

# process the input data
extract_data_for_website(input_data, output_dir, data_page)


## 2. List of resources

In [13]:
# importing data from manually downloaded spreadsheet
input_data = pd.read_csv('data/WP2A2 - Green Skills Resources(Green Skills Resources).csv', encoding = "ISO-8859-1")

In [ ]:

# importing data from manually downloaded spreadsheet
input_data = pd.read_csv('data/WP2A2 - Green Skills Resources(Green Skills Resources).csv')

# location to store the auto-generated qmd files 
output_dir = 'green-skills-resources/auto-generated/' 

# there is an intro page for that spreadsheet
data_page = 'green-skills-resources-list'

# process the input data
extract_data_for_website(input_data, output_dir, data_page)


In [ ]:
input_data


In [10]:
for i in range(len(initiatives)):
    row = initiatives.loc[i,]
    convert_row_to_qmd(row)

In [11]:
# list files in the initiatives folder:
auto_files = os.listdir(output_dir)

In [12]:
# add included statements in the initiative file

text = '' # '## Green Skills Initiatives Repository'
text += '\n\n'

for i in auto_files:
    text += '{{'
    text += f'< include {output_dir}{i} >'
    text += '}}\n\n'

save_qmd(text, 'green-skills-initiatives-list')
